In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [ ]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # Number of features and classes of the original dataset
    num_features = train_X.shape[1]
    num_classes = train_y.shape[1]

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    print("Verbose output only for first run...")
    verbose_run = 1
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            X_train, y_train, 
            epochs=1000, 
            batch_size=32, 
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)

        # Compute metrics
        balanced_acc = balanced_accuracy_score(test_y, y_pred)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")
        

# Shuttle

In [5]:
run_model("shuttle_train.csv", "shuttle_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - accuracy: 0.9338 - loss: 0.3094 - val_accuracy: 0.9945 - val_loss: 0.0324
Epoch 2/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9956 - loss: 0.0283 - val_accuracy: 0.9982 - val_loss: 0.0126
Epoch 3/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9969 - loss: 0.0174 - val_accuracy: 0.9981 - val_loss: 0.0131
Epoch 4/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9975 - loss: 0.0180 - val_accuracy: 0.9981 - val_loss: 0.0061
Epoch 5/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9980 - loss: 0.0102 - val_accuracy: 0.9988 - val_loss: 0.0057
Epoch 6/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9988 - loss: 0.0052 - val_accuracy: 0.9982 - val_loss: 0.0048
Epoch 7/1000
1160/1160 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9982 - loss: 0.0061 - val_accuracy: 0.9991 - val_loss: 0.0055
Epoch 8/1000
1160/1

# Covertype

In [6]:
run_model("covtype_train.csv", "covtype_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 83s 6ms/step - accuracy: 0.5079 - loss: 1.1991 - val_accuracy: 0.5130 - val_loss: 1.1757
Epoch 2/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.5122 - loss: 1.1787 - val_accuracy: 0.5130 - val_loss: 1.1734
Epoch 3/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.5126 - loss: 1.1770 - val_accuracy: 0.5130 - val_loss: 1.1725
Epoch 4/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.5139 - loss: 1.1748 - val_accuracy: 0.5130 - val_loss: 1.1727
Epoch 5/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.5132 - loss: 1.1735 - val_accuracy: 0.5130 - val_loss: 1.1725
Epoch 6/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 66s 6ms/step - accuracy: 0.5142 - loss: 1.1717 - val_accuracy: 0.5130 - val_loss: 1.1723
Epoch 7/1000
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.5118 - loss: 1.1749 - val_accuracy: 0.5130 - val_loss: 1.1724

# KDD

In [7]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 141s 6ms/step - accuracy: 0.9549 - loss: 0.2434 - val_accuracy: 0.9551 - val_loss: 0.2045
Epoch 2/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 122s 6ms/step - accuracy: 0.9550 - loss: 0.2058 - val_accuracy: 0.9551 - val_loss: 0.2042
Epoch 3/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 122s 6ms/step - accuracy: 0.9549 - loss: 0.2053 - val_accuracy: 0.9551 - val_loss: 0.2042
Epoch 4/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 123s 6ms/step - accuracy: 0.9554 - loss: 0.2030 - val_accuracy: 0.9551 - val_loss: 0.2047
Epoch 5/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 125s 6ms/step - accuracy: 0.9551 - loss: 0.2036 - val_accuracy: 0.9551 - val_loss: 0.2045
Epoch 6/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.9548 - loss: 0.2047 - val_accuracy: 0.9551 - val_loss: 0.2043
Epoch 7/1000
20371/20371 ━━━━━━━━━━━━━━━━━━━━ 127s 6ms/step - accuracy: 0.9554 - loss: 0.2035 - val_accuracy: 0.9551 - val_loss:

# Darknet

In [8]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("darknet_train.csv", "darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - accuracy: 0.8374 - loss: 0.6164 - val_accuracy: 0.8466 - val_loss: 0.4861
Epoch 2/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8462 - loss: 0.4993 - val_accuracy: 0.8467 - val_loss: 0.4811
Epoch 3/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8471 - loss: 0.4849 - val_accuracy: 0.8467 - val_loss: 0.4674
Epoch 4/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8449 - loss: 0.4817 - val_accuracy: 0.8467 - val_loss: 0.4624
Epoch 5/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8485 - loss: 0.4666 - val_accuracy: 0.8468 - val_loss: 0.4631
Epoch 6/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8475 - loss: 0.4655 - val_accuracy: 0.8467 - val_loss: 0.4721
Epoch 7/1000
3173/3173 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8475 - loss: 0.4617 - val_accuracy: 0.8467 - val_loss: 0.4578
Epoch 8/1000
